## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np
import numba

In [2]:
%load_ext line_profiler

In [3]:
A = np.random.randint(0,1000, size=(1000000,))
A

array([380, 289, 308, ..., 700, 304, 644])

In [4]:
def f1(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai +100
        acc += bi
        cnt += 1
    return acc/ cnt

In [5]:
%timeit f1(A)

149 ms ± 4.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
def f2(A):
    acc = 0
    for ai in A:
        bi = ai +100
        acc += bi
    return acc / len(A)
        

In [7]:
%timeit f2(A)

127 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%lprun -f f2 f2(A)

In [9]:
%lprun -f f1 f1(A)

In [10]:
def f3(A):
    return A.mean()+100

In [11]:
%timeit f3(A)

337 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%lprun -f f3 f3(A)

In [13]:
@numba.njit
def f5(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai +100
        acc += bi
        cnt += 1
    return acc/cnt

In [14]:
%timeit f5(A)

346 µs ± 3.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [25]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)


def g_optimize(df):
    return df[df["key"].str.contains("a|b|c|d|e")]

def g_optimize_v2(df):
    return df[df["key"].isin(('a', 'b', 'c', 'd', 'e', 'f', 'g'))]
    

In [26]:
%timeit g(df)

368 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit g_optimize(df)

395 ms ± 5.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit g_optimize_v2(df)

59.6 ms ± 300 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

In [20]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [21]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
